In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
pip install google-api-python-client

In [8]:
import json
import requests
from datetime import datetime

In [29]:
# YouTube Data API 키 설정
API_KEY = ""

In [18]:
# 수집할 기준 날짜 (UTC 기준으로 2024-11-05 이전)
CUTOFF_DATE = datetime(2024, 11, 5, 0, 0, 0)

In [19]:
# JSONL 출력 파일명
OUTPUT_FILE = "harris_comments_api.jsonl"

In [20]:
# 수집할 JSON 파일 로드
with open("/content/drive/MyDrive/[인공지능기초]/data/sns/youtube/api/harris/harris_youtube_url_title_date.json", "r", encoding="utf-8") as f:
    videos = json.load(f)

In [21]:
def extract_video_id(url):
    # "watch?v=" 뒤의 값을 추출
    # 예: https://www.youtube.com/watch?v=_v_4IgeVQ8M -> _v_4IgeVQ8M 추출
    # url은 일반적으로 다음 형태: https://www.youtube.com/watch?v=<VIDEO_ID>
    if "watch?v=" in url:
        return url.split("watch?v=")[1].split("&")[0]
    return None

In [22]:
def get_comments(video_id, api_key):
    # commentThreads API를 통해 댓글 조회
    # 하나의 요청으로 최대 100개 가져올 수 있으므로 페이지네이션 처리 필요
    # publishedAt이 cutoff 이전인 것만 필터링
    comments = []
    url = "https://www.googleapis.com/youtube/v3/commentThreads"
    params = {
        "part": "snippet",
        "videoId": video_id,
        "key": api_key,
        "maxResults": 100,
        "textFormat": "plainText",
        "order": "time"
    }

    while True:
        r = requests.get(url, params=params)
        if r.status_code != 200:
            print(f"Error fetching comments for video {video_id}: {r.text}")
            break

        data = r.json()
        items = data.get("items", [])
        if not items:
            break

        for item in items:
            # 최상위 댓글
            top_comment = item["snippet"]["topLevelComment"]["snippet"]
            published_at = top_comment["publishedAt"]  # 예: "2024-10-30T03:12:05Z"
            published_dt = datetime.strptime(published_at, "%Y-%m-%dT%H:%M:%SZ")

            if published_dt < CUTOFF_DATE:
                comments.append({
                    "video_id": video_id,
                    "comment_id": item["snippet"]["topLevelComment"]["id"],
                    "authorDisplayName": top_comment["authorDisplayName"],
                    "textDisplay": top_comment["textDisplay"],
                    "publishedAt": published_at,
                    "likeCount": top_comment.get("likeCount", 0)
                })
            else:
                # 현재 정렬 순서가 time(최신순)라면 여기서부터는 모두 cutoff 이후이므로 중단
                # 하지만 명시적 보장을 위해 전체를 확인할 수도 있음.
                # 일단 넘어가기로 한다.
                pass

        # 다음 페이지 토큰 처리
        if "nextPageToken" in data:
            params["pageToken"] = data["nextPageToken"]
        else:
            break

    return comments


In [23]:
all_comments = []

In [24]:
print("총 영상 수:", len(videos))

총 영상 수: 126


In [25]:
for idx, video_info in enumerate(videos, start=1):
    vid_id = extract_video_id(video_info["url"])
    if not vid_id:
        print(f"[{idx}/{len(videos)}] 영상 URL에서 ID를 추출할 수 없습니다: {video_info['url']}")
        continue

    print(f"[{idx}/{len(videos)}] 영상(ID: {vid_id}) 댓글 수집 중...")
    video_comments = get_comments(vid_id, API_KEY)
    print(f" -> 수집된 댓글 수: {len(video_comments)}")
    all_comments.extend(video_comments)

[1/126] 영상(ID: _v_4IgeVQ8M) 댓글 수집 중...
 -> 수집된 댓글 수: 142
[2/126] 영상(ID: t9suRUZGIdg) 댓글 수집 중...
 -> 수집된 댓글 수: 157
[3/126] 영상(ID: OwFWgewAAXc) 댓글 수집 중...
 -> 수집된 댓글 수: 76
[4/126] 영상(ID: Cb0AothNHiw) 댓글 수집 중...
 -> 수집된 댓글 수: 181
[5/126] 영상(ID: oU9IVXyGBYI) 댓글 수집 중...
 -> 수집된 댓글 수: 655
[6/126] 영상(ID: ShwoUTax4B0) 댓글 수집 중...
 -> 수집된 댓글 수: 217
[7/126] 영상(ID: icMONoEE61M) 댓글 수집 중...
 -> 수집된 댓글 수: 277
[8/126] 영상(ID: Wo7KwWIg-7o) 댓글 수집 중...
 -> 수집된 댓글 수: 935
[9/126] 영상(ID: U6bv6jYEVAs) 댓글 수집 중...
 -> 수집된 댓글 수: 1105
[10/126] 영상(ID: 81HpMyoXd0g) 댓글 수집 중...
 -> 수집된 댓글 수: 274
[11/126] 영상(ID: -XINcrcvrMU) 댓글 수집 중...
 -> 수집된 댓글 수: 1929
[12/126] 영상(ID: gL0IFZk44hM) 댓글 수집 중...
 -> 수집된 댓글 수: 172
[13/126] 영상(ID: 9W826GfFwak) 댓글 수집 중...
 -> 수집된 댓글 수: 2932
[14/126] 영상(ID: pWdwTgxGl_U) 댓글 수집 중...
 -> 수집된 댓글 수: 1228
[15/126] 영상(ID: kBqPu-FetF4) 댓글 수집 중...
 -> 수집된 댓글 수: 361
[16/126] 영상(ID: 3_Z7ucCx2_w) 댓글 수집 중...
 -> 수집된 댓글 수: 378
[17/126] 영상(ID: GRtF4lwBTgQ) 댓글 수집 중...
 -> 수집된 댓글 수: 200
[18/126] 영상(ID: zGrP

In [26]:
print(f"총 수집된 댓글 수: {len(all_comments)}")

총 수집된 댓글 수: 96987


In [27]:
# jsonl로 저장
print("jsonl 파일로 저장 중...")
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for comment in all_comments:
        f.write(json.dumps(comment, ensure_ascii=False) + "\n")

jsonl 파일로 저장 중...


In [28]:
print("저장 완료:", OUTPUT_FILE)

저장 완료: harris_comments_api.jsonl
